# Teenage Pandas
In this intermediate course on Pandas, we're going to continue analyzing our beer data. This is intentional so as to allow you to clearly see how we have increased the complexity in the data preprocessing and analysis here as compared to the baby_pandas course. 

## Getting Started
Let's start with the basics.
- Import statements
- Loading the data into a Pandas DataFrame
- Observing the first 5 values of the DataFrame

In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('Cans of Beer Sold.csv')

In [5]:
df.head()

,Date,Year,Temperature,Cans of beer sold
0,1-Jun,2010,71,9150
1,20-Jun,2010,81,10084
2,12-Jul,2010,71,9242
3,28-Jul,2010,83,10361
4,3-Aug,2010,65,8829


## Data Preprocessing
We performed some interesting EDA in the previous course and even scratched the surface of Data Preprocessing when we converted _Year_ into a categorical variable. 

We didn't touch upon Date

We know from common sense that _Date_ would be better represented as a `datetime` datatype. Converting _Date_ from `object` to `datetime` is known as **Type Casting** because we cast the datatype from its previous value to a new value. The biggest advantage of representing the _Date_ column as `datetime` is to be able to plot a time series against this dataset. We'll cover this in the Teenage Pandas course. 

## Pandas Series
Pandas offers a structure to hold one-dimensional data known as a Series. A Pandas Series can be thought of as a single column from an Excel spreadsheet, capable of holding data of any datatype (strings, integers, floats, datetimes, categorical values etc). 

Similar to a DataFrame, a Series can have a name as well as axis labels known as an index. (**Check in notebook**) 

> We can obtain multiple Series pertaining to each column in our DataFrame above. For example, if we were to extract the _Date_ column, it would be stored as a Pandas Series. We're going to see this in action really soon!

### Once again, we're back to the same question... how do we access a particular column in the DataFrame?
1. Using the square bracket notation - we've already seen this in the previous course. 
2. Using the dot notation - typing the DataFrame name followed by a dot followed by the column name. This is the one we're about to use. 